In [1]:
# Web Scraper

# Imports
import os  #manipular o sistema operacional
import re  #expressão regular que usa-se pra buscar padrões nos dados
import csv #pra gravar em csv o resultado
import pickle #para criar um cache de acesso a pagina web
import requests # para fazer requisição a pagina
from bs4 import BeautifulSoup # pra fazer o parser do código html
from urllib.request import urlopen
from pprint import pprint
import urllib3
import pandas as pd


In [2]:
url = 'https://lista.mercadolivre.com.br/celular#D[A:celular]'

results = requests.get(url)

texto_web = results.text

In [14]:
soup = BeautifulSoup(texto_web,'html.parser')

In [28]:
def get_link_pages(soup):
    try:
        links = soup.findAll('a', class_='andes-pagination__link ui-search-link')
        urls = [item.get('href') for item in links]
    except:
        linK = 'não foi possível'
    return urls
get_link_pages(soup)    

['https://celulares.mercadolivre.com.br/',
 'https://celulares.mercadolivre.com.br/_Desde_51',
 'https://celulares.mercadolivre.com.br/_Desde_101',
 'https://celulares.mercadolivre.com.br/_Desde_151',
 'https://celulares.mercadolivre.com.br/_Desde_201',
 'https://celulares.mercadolivre.com.br/_Desde_251',
 'https://celulares.mercadolivre.com.br/_Desde_301',
 'https://celulares.mercadolivre.com.br/_Desde_351',
 'https://celulares.mercadolivre.com.br/_Desde_401',
 'https://celulares.mercadolivre.com.br/_Desde_451',
 'https://celulares.mercadolivre.com.br/_Desde_51']

In [92]:

url = 'https://lista.mercadolivre.com.br/celular#D[A:celular]'

results = requests.get(url)

texto_web = results.text

soup = BeautifulSoup(texto_web,'html.parser')

urls = get_link_pages(soup) 

bloco_mercado_livre = []
for link in urls:    
    page = link

    results = requests.get(page)

    texto_web = results.text

    soup = BeautifulSoup(texto_web,'html.parser')

    
    for info in soup.findAll('div', {'class':'ui-search-result__content-wrapper'}):
        bloco = []

        # find
        # Nome e específicações
        info_nome = info.find('a').attrs['title']
        # Preço
        info_cent = info.find('span', {'class':'price-tag-fraction'})
        # Ram do aparelho
        try:
            info_ram = re.findall(r'\d+?[\s+\d+][gGmM][bB][\s+\w][rR][aA][mM]', info_nome, re.M|re.I)
        except:
        
        info_sem_ram = re.sub(r'\d+?[\s+\d+][gGmM][bB][\s+\w][rR][aA][mM]', "", info_nome, re.M|re.I)
        info_sem_marca = re.sub(r'^\w+', "" ,info_sem_ram, re.M|re.I)
        info_dual_sim = re.findall(r'Dual\sSIM',info_sem_marca, re.M|re.I)
        if info_dual_sim:
            info_dual_sim = ['2']
        else:
            info_dual_sim = ['1']



        # findall
        info_nome_marca = re.findall(r'^\w+', info_nome, re.M|re.I)
        
            
        




        info_sem_dual = re.sub(r'Dual\sSIM', "" ,info_sem_marca, re.M|re.I)
        info_hd = re.findall(r'\d+[\w\s][gGmM][bB]',info_sem_dual, re.M|re.I )
        info_sem_hd = re.sub(r'\d+[\w\s][gGmB][bB]', "", info_sem_dual, re.M|re.I )
        info_cor = re.findall(r'\w+[-]?\w+[\s]?$',info_sem_hd, re.M|re.I )
        info_sem_cor = re.sub(r'\w+[-]?\w+[\s]?$', "",info_sem_hd, re.M|re.I )

        # Nome e específicações
    #     bloco.append(str(info_nome))
    #     print('Nome e específicações:', info_nome)
    
        bloco.append(info_nome)
        print('info:', info_nome)

        # Informações sem ram, marca, dual e hd
        bloco.append(info_sem_cor)
        print('Nome do Celular:', info_sem_cor)


        # Marca do celular
        bloco.extend(info_nome_marca)
        print('Marca do celular:', info_nome_marca)


        # Cor do Celular
        bloco.extend(info_cor)
        print('Cor:', info_cor)


        # Memória ram
        bloco.extend(info_ram)
        print('Memória ram:', info_ram)

        # Espaço de hd
        bloco.extend(info_hd)
        print('Espaço em hd:', info_hd)


        # 1/2 ship(s)
        bloco.extend(info_dual_sim)
        print('1 ou 2 ship(s):', info_dual_sim)


        # Preço do celualr
        bloco.append(info_cent.text)
        print('Preço:', info_cent.text)
        print('=======================================')

        # lista mestre
        bloco_mercado_livre.append(bloco)

    #     bloco_mercado_livre.append([info_nome,info_nome_marca, info_dual_sim,
    #                                 str(info_cent).join(info_val)+str(info_val).join(info_cent)])
    #     pprint(bloco_mercado_livre)


    #     print(info_nome)
    #     print(info_nome_re.group(1))
    #     print(info_nome_re.group(2))
    #     print(info_dual_sim)
    #     print( '===========')


    # print(len(bloco_mercado_livre))        
      


info: LG K52 (13 Mpx) Dual SIM 64 GB  red 3 GB RAM
Nome do Celular:  K52 (13 Mpx)    
Marca do celular: ['LG']
Cor: ['red ']
Memória ram: ['6 GB RAM']
Espaço em hd: ['64 GB']
1 ou 2 ship(s): ['2']
Preço: 1.070
info: Samsung Galaxy A31 Dual SIM 128 GB  prism crush black 4 GB RAM
Nome do Celular:  Galaxy A31    prism crush 
Marca do celular: ['Samsung']
Cor: ['black ']
Memória ram: ['6 GB RAM']
Espaço em hd: ['128 GB']
1 ou 2 ship(s): ['2']
Preço: 1.537
info: Samsung Galaxy A01 Core Dual SIM 32 GB  azul 2 GB RAM
Nome do Celular:  Galaxy A01 Core    
Marca do celular: ['Samsung']
Cor: ['azul ']
Memória ram: ['6 GB RAM']
Espaço em hd: ['32 GB']
1 ou 2 ship(s): ['2']
Preço: 688
info: Samsung Galaxy A10s Dual SIM 32 GB  azul 2 GB RAM
Nome do Celular:  Galaxy A10s    
Marca do celular: ['Samsung']
Cor: ['azul ']
Memória ram: ['6 GB RAM']
Espaço em hd: ['32 GB']
1 ou 2 ship(s): ['2']
Preço: 838
info: Samsung Galaxy A21s Dual SIM 64 GB preto 4 GB RAM
Nome do Celular:  Galaxy A21s   
Marca do ce

info: Smartphone Samsung Galaxy M21s Tela 6.4 64gb 4gb Ram - Azul
Nome do Celular:  Samsung Galaxy M21s Tela 6.4  4gb Ram - 
Marca do celular: ['Smartphone']
Cor: ['Azul']
Memória ram: ['6 GB RAM']
Espaço em hd: ['64gb']
1 ou 2 ship(s): ['1']
Preço: 1.499
info: Celular Asus Zb634kl Zenfone Max Shot Plus 128gb (64gb+64gb)
Nome do Celular:  Asus Zb634kl Zenfone Max Shot Plus  (+)
Marca do celular: ['Celular']
Cor: []
Memória ram: ['6 GB RAM']
Espaço em hd: ['128gb', '64gb', '64gb']
1 ou 2 ship(s): ['1']
Preço: 999
info: Smartphone LG K22 Tela 6,2 32gb 2gb Ram Android Azul
Nome do Celular:  LG K22 Tela 6,2  2gb Ram Android 
Marca do celular: ['Smartphone']
Cor: ['Azul']
Memória ram: ['6 GB RAM']
Espaço em hd: ['32gb']
1 ou 2 ship(s): ['1']
Preço: 741
info: Smartphone LG K22 Plus Tela 6,2 64gb 3gb Ram Android Titânio
Nome do Celular:  LG K22 Plus Tela 6,2  3gb Ram Android 
Marca do celular: ['Smartphone']
Cor: ['Titânio']
Memória ram: ['6 GB RAM']
Espaço em hd: ['64gb']
1 ou 2 ship(s): ['1

1 ou 2 ship(s): ['1']
Preço: 1.899
info: Celular Flip Vita Dourado Dual Chip Quadriband P9043 Multila
Nome do Celular:  Flip Vita Dourado Dual Chip Quadriband P9043 
Marca do celular: ['Celular']
Cor: ['Multila']
Memória ram: ['6 GB RAM']
Espaço em hd: []
1 ou 2 ship(s): ['1']
Preço: 185
info: Moto E6 Play Dual SIM 32 GB  cinza-metálico 2 GB RAM
Nome do Celular:  E6 Play    
Marca do celular: ['Moto']
Cor: ['cinza-metálico ']
Memória ram: ['6 GB RAM']
Espaço em hd: ['32 GB']
1 ou 2 ship(s): ['2']
Preço: 888
info: Xiaomi Mi 10T 5G Dual SIM 128 GB  prata-lunar 6 GB RAM
Nome do Celular:  Mi 10T 5G    
Marca do celular: ['Xiaomi']
Cor: ['prata-lunar ']
Memória ram: ['6 GB RAM']
Espaço em hd: ['128 GB']
1 ou 2 ship(s): ['2']
Preço: 3.500
info: Motorola One Hyper Dual SIM 128 GB  deepsea blue 4 GB RAM
Nome do Celular:  One Hyper    deepsea 
Marca do celular: ['Motorola']
Cor: ['blue ']
Memória ram: ['6 GB RAM']
Espaço em hd: ['128 GB']
1 ou 2 ship(s): ['2']
Preço: 1.920
info: Xiaomi Mi 10 Li

info: Celular Samsung Galaxy A71 A715 Dual 128gb 6gb Ram Preto
Nome do Celular:  Samsung Galaxy A71 A715 Dual  6gb Ram 
Marca do celular: ['Celular']
Cor: ['Preto']
Memória ram: ['6 GB RAM']
Espaço em hd: ['128gb']
1 ou 2 ship(s): ['1']
Preço: 2.189
info: Celular Moto G8 Power Lite 64gb Câmera Tripl Tela 6.5 Aqua
Nome do Celular:  Moto G8 Power Lite  Câmera Tripl Tela 6.5 
Marca do celular: ['Celular']
Cor: ['Aqua']
Memória ram: ['6 GB RAM']
Espaço em hd: ['64gb']
1 ou 2 ship(s): ['1']
Preço: 1.170
info: Smartphone Samsung Galaxy S10 Tela De 6.1 128gb 8gb Ram
Nome do Celular:  Samsung Galaxy S10 Tela De 6.1  8gb 
Marca do celular: ['Smartphone']
Cor: ['Ram']
Memória ram: ['6 GB RAM']
Espaço em hd: ['128gb']
1 ou 2 ship(s): ['1']
Preço: 2.799
info: Smartphone LG K22 Plus Tela 6,2 64gb 3gb Ram Quad-core Azul
Nome do Celular:  LG K22 Plus Tela 6,2  3gb Ram Quad-core 
Marca do celular: ['Smartphone']
Cor: ['Azul']
Memória ram: ['6 GB RAM']
Espaço em hd: ['64gb']
1 ou 2 ship(s): ['1']
Preço

info: Smartphone LG K22 Tela 6,2 32gb 2gb Ram Android Titânio
Nome do Celular:  LG K22 Tela 6,2  2gb Ram Android 
Marca do celular: ['Smartphone']
Cor: ['Titânio']
Memória ram: ['6 GB RAM']
Espaço em hd: ['32gb']
1 ou 2 ship(s): ['1']
Preço: 999
info: Smartphone Samsung Galaxy M31 Tela 6.4 128gb 6gb Ram - Azul
Nome do Celular:  Samsung Galaxy M31 Tela 6.4  6gb Ram - 
Marca do celular: ['Smartphone']
Cor: ['Azul']
Memória ram: ['6 GB RAM']
Espaço em hd: ['128gb']
1 ou 2 ship(s): ['1']
Preço: 1.799
info: Celular Samsung Galaxy A31 A315 Dual 128gb 4gb Ram Azul
Nome do Celular:  Samsung Galaxy A31 A315 Dual  4gb Ram 
Marca do celular: ['Celular']
Cor: ['Azul']
Memória ram: ['6 GB RAM']
Espaço em hd: ['128gb']
1 ou 2 ship(s): ['1']
Preço: 1.569
info: Smartphone Samsung Galaxy A01 Core Tela 5.3 32gb 2gb Ram Pto
Nome do Celular:  Samsung Galaxy A01 Core Tela 5.3  2gb Ram 
Marca do celular: ['Smartphone']
Cor: ['Pto']
Memória ram: ['6 GB RAM']
Espaço em hd: ['32gb']
1 ou 2 ship(s): ['1']
Preço

...

Nome do Celular:  Redmi Note 8 Pro Dual Sim  Verde-floresta  
Marca do celular: ['Xiaomi']
Cor: []
Memória ram: ['6 GB RAM']
Espaço em hd: ['128 Gb', '6 Gb']
1 ou 2 ship(s): ['2']
Preço: 1.605
info: Smartphone Samsung Galaxy A51 128gb 6.5
Nome do Celular:  Samsung Galaxy A51  6.5
Marca do celular: ['Smartphone']
Cor: []
Memória ram: ['6 GB RAM']
Espaço em hd: ['128gb']
1 ou 2 ship(s): ['1']
Preço: 2.758
info: Smartphone Samsung Galaxy A31 - 128 Gb
Nome do Celular:  Samsung Galaxy A31 - 
Marca do celular: ['Smartphone']
Cor: []
Memória ram: ['6 GB RAM']
Espaço em hd: ['128 Gb']
1 ou 2 ship(s): ['1']
Preço: 1.699
info: Smartphone LG K62 4g Tela 6,6 64gb 4gb Ram Azul
Nome do Celular:  LG K62 4g Tela 6,6  4gb Ram 
Marca do celular: ['Smartphone']
Cor: ['Azul']
Memória ram: ['6 GB RAM']
Espaço em hd: ['64gb']
1 ou 2 ship(s): ['1']
Preço: 1.199
info: Celular Telefone Idoso Whatsapp Multilaser Zapp 3g -p9098
Nome do Celular:  Telefone Idoso Whatsapp Multilaser Zapp 3g -
Marca do celular: ['Ce

info: Xiaomi Redmi 9i 128gb + 4gb Ram Vglobal Bateria Maior C/nota
Nome do Celular:  Redmi 9i  + 4gb Ram Vglobal Bateria Maior C/
Marca do celular: ['Xiaomi']
Cor: ['nota']
Memória ram: ['6 GB RAM']
Espaço em hd: ['128gb']
1 ou 2 ship(s): ['1']
Preço: 1.190
info: Xiaomi Redmi Note 9 Pro 128gb 6gb Ram Versao Global+ Nota
Nome do Celular:  Redmi Note 9 Pro  6gb Ram Versao Global+ 
Marca do celular: ['Xiaomi']
Cor: ['Nota']
Memória ram: ['6 GB RAM']
Espaço em hd: ['128gb']
1 ou 2 ship(s): ['1']
Preço: 2.068
info: Smartphone Positivo Twist 3 S513 32gb Tela 5.5 Polegadas
Nome do Celular:  Positivo Twist 3 S513  Tela 5.5 
Marca do celular: ['Smartphone']
Cor: ['Polegadas']
Memória ram: ['6 GB RAM']
Espaço em hd: ['32gb']
1 ou 2 ship(s): ['1']
Preço: 793
info: iPhone 12 mini 128 GB  (product)red
Nome do Celular:  12 mini   (product)
Marca do celular: ['iPhone']
Cor: ['red']
Memória ram: ['6 GB RAM']
Espaço em hd: ['128 GB']
1 ou 2 ship(s): ['1']
Preço: 6.349
info: Smartphone Semp Tcl L5 16gb 

info: Smatphone Multilaser Dois Chip 32 Gb Melhor Q Oba Fone Idoso
Nome do Celular:  Multilaser Dois Chip  Melhor Q Oba Fone 
Marca do celular: ['Smatphone']
Cor: ['Idoso']
Memória ram: ['6 GB RAM']
Espaço em hd: ['32 Gb']
1 ou 2 ship(s): ['1']
Preço: 959
info: Samsung Galaxy A12 Dual 6.5 64gb 4gb Ram Garantia De 1 Ano
Nome do Celular:  Galaxy A12 Dual 6.5  4gb Ram Garantia De 1 
Marca do celular: ['Samsung']
Cor: ['Ano']
Memória ram: ['6 GB RAM']
Espaço em hd: ['64gb']
1 ou 2 ship(s): ['1']
Preço: 1.218
info: Xiaomi Redmi 9t 128gb 4g Ram - Versão Global
Nome do Celular:  Redmi 9t  4g Ram - Versão 
Marca do celular: ['Xiaomi']
Cor: ['Global']
Memória ram: ['6 GB RAM']
Espaço em hd: ['128gb']
1 ou 2 ship(s): ['1']
Preço: 1.449
info: Celular Do Idoso 32gb Letras Grandes Tela 4' Lite Smart Oba
Nome do Celular:  Do Idoso  Letras Grandes Tela 4' Lite Smart 
Marca do celular: ['Celular']
Cor: ['Oba']
Memória ram: ['6 GB RAM']
Espaço em hd: ['32gb']
1 ou 2 ship(s): ['1']
Preço: 469
info: Xiao

info: Galaxy A01 Core 32 Gb Preto - Cx. Lacrada - Garantia Samsung
Nome do Celular:  A01 Core  Preto - Cx. Lacrada - Garantia 
Marca do celular: ['Galaxy']
Cor: ['Samsung']
Memória ram: ['6 GB RAM']
Espaço em hd: ['32 Gb']
1 ou 2 ship(s): ['1']
Preço: 629
info: Xiaomi Redmi 9 Dual Sim 128 Gb Ocean Green 4 Gb Ram
Nome do Celular:  Redmi 9 Dual Sim  Ocean 
Marca do celular: ['Xiaomi']
Cor: ['Green ']
Memória ram: ['6 GB RAM']
Espaço em hd: ['128 Gb']
1 ou 2 ship(s): ['2']
Preço: 1.269
info:  iPhone 8 Plus Vitrine 64gb Original Apple Em  12x Sem Juros
Nome do Celular:  iPhone 8 Plus Vitrine  Original Apple Em  12x Sem 
Marca do celular: []
Cor: ['Juros']
Memória ram: ['6 GB RAM']
Espaço em hd: ['64gb']
1 ou 2 ship(s): ['1']
Preço: 2.999
info: Xiaomi Redmi Note 9 Dual Sim 128 Gb 4 Ram Com Nota Fiscal 
Nome do Celular:  Redmi Note 9 Dual Sim  4 Ram Com Nota 
Marca do celular: ['Xiaomi']
Cor: ['Fiscal ']
Memória ram: ['6 GB RAM']
Espaço em hd: ['128 Gb']
1 ou 2 ship(s): ['2']
Preço: 1.449
in

Marca do celular: ['Multilaser']
Cor: ['preto ']
Memória ram: ['6 GB RAM']
Espaço em hd: ['32 GB']
1 ou 2 ship(s): ['2']
Preço: 416
info: Positivo Twist S511 Dual SIM 16 GB  cinza 1 GB RAM
Nome do Celular:  Twist S511    
Marca do celular: ['Positivo']
Cor: ['cinza ']
Memória ram: ['6 GB RAM']
Espaço em hd: ['16 GB']
1 ou 2 ship(s): ['2']
Preço: 453
info: Xiaomi Redmi 9 (Global) Dual SIM 128 GB ocean green 4 GB RAM
Nome do Celular:  Redmi 9 (Global)   ocean 
Marca do celular: ['Xiaomi']
Cor: ['green ']
Memória ram: ['6 GB RAM']
Espaço em hd: ['128 GB']
1 ou 2 ship(s): ['2']
Preço: 1.330
info: iPhone 12 128 GB  azul
Nome do Celular:  12   
Marca do celular: ['iPhone']
Cor: ['azul']
Memória ram: ['6 GB RAM']
Espaço em hd: ['128 GB']
1 ou 2 ship(s): ['1']
Preço: 7.040
info: Positivo Twist 3 Fit S509C Dual SIM 32 GB  ouro 512 MB RAM
Nome do Celular:  Twist 3 Fit S509C    
Marca do celular: ['Positivo']
Cor: ['ouro ']
Memória ram: ['6 GB RAM']
Espaço em hd: ['32 GB']
1 ou 2 ship(s): ['2']
P

In [42]:
df_cel_mercado = pd.DataFrame(bloco_mercado_livre, columns=['Nome', 'Marca','Cor','Memória_ram', 'Espaço_em_HD' ,'um_ou_2_ships', 'Valor(R$)', 'x','x2'])

In [43]:
df_cel_mercado.head()

,Nome,Marca,Cor,Memória_ram,Espaço_em_HD,um_ou_2_ships,Valor(R$),x,x2
0,K52 (13 Mpx),LG,red,3 GB RAM,64 GB,2,1.070,None,None
1,Redmi Note 9S,Xiaomi,azul,4 GB RAM,128 GB,2,1.709,None,None
2,Galaxy A31 prism crush,Samsung,black,4 GB RAM,128 GB,2,1.537,None,None
3,Galaxy A01 Core,Samsung,azul,2 GB RAM,32 GB,2,779,None,None
4,ZenFone Max Shot ZB634KL,Asus,preto,4 GB RAM,64 GB,2,999,None,None


In [44]:
df_cel_mercado.shape

(609, 9)

In [85]:
len(df_num_nome)

359

In [88]:
df_num_nome = df_cel_mercado.Nome.value_counts()
df_num_nome = pd.DataFrame(df_num_nome, columns=['Nome'])
df_num_nome_maior_1 = df_num_nome.query('Nome >1')
df_num_nome_maior_1

,Nome
"LG K22 Tela 6,2 2gb Ram Android",8
"LG K52 4g Tela 6,59 3gb Ram",8
Samsung Galaxy A31 A315 Dual 4gb Ram,7
Samsung Galaxy A10s Tela De 6.2 2gb,7
Samsung Galaxy A12 Tela 6.5 4gb Ram,7
...,...
iPhone 8 Vitrine Dourado Garantia +,2
Galaxy A01 Core,2
Nokia 2.3 Nk005 Dual 4g 2gb Ram Hd+ 6.2,2
ZenFone Max Shot ZB634KL,2


In [38]:
lista_1 = ['R$ 9.900']
lista_2 = [',99']
str(lista_1).join(lista_2).replace(',','.')

'.99'

In [ ]:
str(lista_2).join(lista_1).replace('.',',')

In [ ]:
print(str(lista_2).join(lista_1)+str(lista_1).join(lista_2))

In [ ]:
bloco_zoom = []

for info in soup.findAll('div', {'class':'cardBody'}):
#     bloco = []
    
    info_nome = info.find('a').attrs['title']
    
    info_val = info.find('span', {'class':'mainValue'})
    
    info_cent = info.find('span', {'class':'centsValue'})
    
    info_nome_re = re.match(r'^\w+\s(\w+.)(\w+\s+\w+)', info_nome, re.M|re.I)

    info_val = info.find('span', {'class':'mainValue'})
  
    info_val_re = re.findall('(R[$]\s?[+-]?[0-9]{1,3}(?:(?:,?[0-9]{3}))*(?:\.[0-9]{1,2}\d+)?)', str(info_val))
    
    info_cent = info.find('span', {'class':'centsValue'})
    
    info_cent_re = re.findall('(.\s?[+-]?[0-9]{1,3}(?:(?:,?[0-9]{3}))*(?:\.[0-9]{1,2}\d+)?)', str(info_cent))
    bloco_zoom.append([info_nome,info_nome_re.group(1),info_nome_re.group(2),
                  str(info_cent_re).join(info_val_re)+str(info_val_re).join(info_cent_re)])
    
#     print(bloco)
    
#     print(info)
#     print(info_nome)
#     print(info_nome_re.group(1))
#     print(info_nome_re.group(2))
#     print(str(info_cent_re).join(info_val_re)+str(info_val_re).join(info_cent_re))

#     print('=============')
# print(valor)
# info_name = info.find('a').attrs['title']
# pprint(info_name)
print(len(bloco_zoom))
pprint(bloco_zoom)


In [ ]:
# Example of w+ and ^ Expression
import re
xx = "guru99,education is fun"
r1 = re.findall(r"^\w+",xx)
print (r1)

In [ ]:
# Example of \s expression in re.split function
import re
xx = "guru99,education is fun"
r1 = re.findall(r"^\w+", xx)
print (re.split(r'\s','we are splitting the words'))
print (re.split(r's','split the words'))

In [ ]:
# Example of re.M or Multiline Flags
import re
xx = """Guru99 gagre; 
careerguru99	
selenium"""
k1 = re.findall(r"^\w", xx)
k2 = re.findall(r"^\w", xx, re.MULTILINE)
k3 = re.findall("\\bg\w+\\b", xx)
k4 = re.findall("g\w+", xx)
k5 = re.findall("\\bg\w+\\b", xx, flags=re.I)

print (k1)
print (k2)
print (k3)
print (k4)
print (k5)

In [ ]:
date             = re.compile('(?:(?<!\:)(?<!\:\d)[0-3]?\d(?:st|nd|rd|th)?\s+(?:of\s+)?(?:jan\.?|january|feb\.?|february|mar\.?|march|apr\.?|april|may|jun\.?|june|jul\.?|july|aug\.?|august|sep\.?|september|oct\.?|october|nov\.?|november|dec\.?|december)|(?:jan\.?|january|feb\.?|february|mar\.?|march|apr\.?|april|may|jun\.?|june|jul\.?|july|aug\.?|august|sep\.?|september|oct\.?|october|nov\.?|november|dec\.?|december)\s+(?<!\:)(?<!\:\d)[0-3]?\d(?:st|nd|rd|th)?)(?:\,)?\s*(?:\d{4})?|[0-3]?\d[-\./][0-3]?\d[-\./]\d{2,4}', re.IGNORECASE)
time             = re.compile('\d{1,2}:\d{2} ?(?:[ap]\.?m\.?)?|\d[ap]\.?m\.?', re.IGNORECASE)
phone            = re.compile('''((?:(?<![\d-])(?:\+?\d{1,3}[-.\s*]?)?(?:\(?\d{3}\)?[-.\s*]?)?\d{3}[-.\s*]?\d{4}(?![\d-]))|(?:(?<![\d-])(?:(?:\(\+?\d{2}\))|(?:\+?\d{2}))\s*\d{2}\s*\d{3}\s*\d{4}(?![\d-])))''')
phones_with_exts = re.compile('((?:(?:\+?1\s*(?:[.-]\s*)?)?(?:\(\s*(?:[2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|(?:[2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?(?:[2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?(?:[0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(?:\d+)?))', re.IGNORECASE)
link             = re.compile('(?i)((?:https?://|www\d{0,3}[.])?[a-z0-9.\-]+[.](?:(?:international)|(?:construction)|(?:contractors)|(?:enterprises)|(?:photography)|(?:immobilien)|(?:management)|(?:technology)|(?:directory)|(?:education)|(?:equipment)|(?:institute)|(?:marketing)|(?:solutions)|(?:builders)|(?:clothing)|(?:computer)|(?:democrat)|(?:diamonds)|(?:graphics)|(?:holdings)|(?:lighting)|(?:plumbing)|(?:training)|(?:ventures)|(?:academy)|(?:careers)|(?:company)|(?:domains)|(?:florist)|(?:gallery)|(?:guitars)|(?:holiday)|(?:kitchen)|(?:recipes)|(?:shiksha)|(?:singles)|(?:support)|(?:systems)|(?:agency)|(?:berlin)|(?:camera)|(?:center)|(?:coffee)|(?:estate)|(?:kaufen)|(?:luxury)|(?:monash)|(?:museum)|(?:photos)|(?:repair)|(?:social)|(?:tattoo)|(?:travel)|(?:viajes)|(?:voyage)|(?:build)|(?:cheap)|(?:codes)|(?:dance)|(?:email)|(?:glass)|(?:house)|(?:ninja)|(?:photo)|(?:shoes)|(?:solar)|(?:today)|(?:aero)|(?:arpa)|(?:asia)|(?:bike)|(?:buzz)|(?:camp)|(?:club)|(?:coop)|(?:farm)|(?:gift)|(?:guru)|(?:info)|(?:jobs)|(?:kiwi)|(?:land)|(?:limo)|(?:link)|(?:menu)|(?:mobi)|(?:moda)|(?:name)|(?:pics)|(?:pink)|(?:post)|(?:rich)|(?:ruhr)|(?:sexy)|(?:tips)|(?:wang)|(?:wien)|(?:zone)|(?:biz)|(?:cab)|(?:cat)|(?:ceo)|(?:com)|(?:edu)|(?:gov)|(?:int)|(?:mil)|(?:net)|(?:onl)|(?:org)|(?:pro)|(?:red)|(?:tel)|(?:uno)|(?:xxx)|(?:ac)|(?:ad)|(?:ae)|(?:af)|(?:ag)|(?:ai)|(?:al)|(?:am)|(?:an)|(?:ao)|(?:aq)|(?:ar)|(?:as)|(?:at)|(?:au)|(?:aw)|(?:ax)|(?:az)|(?:ba)|(?:bb)|(?:bd)|(?:be)|(?:bf)|(?:bg)|(?:bh)|(?:bi)|(?:bj)|(?:bm)|(?:bn)|(?:bo)|(?:br)|(?:bs)|(?:bt)|(?:bv)|(?:bw)|(?:by)|(?:bz)|(?:ca)|(?:cc)|(?:cd)|(?:cf)|(?:cg)|(?:ch)|(?:ci)|(?:ck)|(?:cl)|(?:cm)|(?:cn)|(?:co)|(?:cr)|(?:cu)|(?:cv)|(?:cw)|(?:cx)|(?:cy)|(?:cz)|(?:de)|(?:dj)|(?:dk)|(?:dm)|(?:do)|(?:dz)|(?:ec)|(?:ee)|(?:eg)|(?:er)|(?:es)|(?:et)|(?:eu)|(?:fi)|(?:fj)|(?:fk)|(?:fm)|(?:fo)|(?:fr)|(?:ga)|(?:gb)|(?:gd)|(?:ge)|(?:gf)|(?:gg)|(?:gh)|(?:gi)|(?:gl)|(?:gm)|(?:gn)|(?:gp)|(?:gq)|(?:gr)|(?:gs)|(?:gt)|(?:gu)|(?:gw)|(?:gy)|(?:hk)|(?:hm)|(?:hn)|(?:hr)|(?:ht)|(?:hu)|(?:id)|(?:ie)|(?:il)|(?:im)|(?:in)|(?:io)|(?:iq)|(?:ir)|(?:is)|(?:it)|(?:je)|(?:jm)|(?:jo)|(?:jp)|(?:ke)|(?:kg)|(?:kh)|(?:ki)|(?:km)|(?:kn)|(?:kp)|(?:kr)|(?:kw)|(?:ky)|(?:kz)|(?:la)|(?:lb)|(?:lc)|(?:li)|(?:lk)|(?:lr)|(?:ls)|(?:lt)|(?:lu)|(?:lv)|(?:ly)|(?:ma)|(?:mc)|(?:md)|(?:me)|(?:mg)|(?:mh)|(?:mk)|(?:ml)|(?:mm)|(?:mn)|(?:mo)|(?:mp)|(?:mq)|(?:mr)|(?:ms)|(?:mt)|(?:mu)|(?:mv)|(?:mw)|(?:mx)|(?:my)|(?:mz)|(?:na)|(?:nc)|(?:ne)|(?:nf)|(?:ng)|(?:ni)|(?:nl)|(?:no)|(?:np)|(?:nr)|(?:nu)|(?:nz)|(?:om)|(?:pa)|(?:pe)|(?:pf)|(?:pg)|(?:ph)|(?:pk)|(?:pl)|(?:pm)|(?:pn)|(?:pr)|(?:ps)|(?:pt)|(?:pw)|(?:py)|(?:qa)|(?:re)|(?:ro)|(?:rs)|(?:ru)|(?:rw)|(?:sa)|(?:sb)|(?:sc)|(?:sd)|(?:se)|(?:sg)|(?:sh)|(?:si)|(?:sj)|(?:sk)|(?:sl)|(?:sm)|(?:sn)|(?:so)|(?:sr)|(?:st)|(?:su)|(?:sv)|(?:sx)|(?:sy)|(?:sz)|(?:tc)|(?:td)|(?:tf)|(?:tg)|(?:th)|(?:tj)|(?:tk)|(?:tl)|(?:tm)|(?:tn)|(?:to)|(?:tp)|(?:tr)|(?:tt)|(?:tv)|(?:tw)|(?:tz)|(?:ua)|(?:ug)|(?:uk)|(?:us)|(?:uy)|(?:uz)|(?:va)|(?:vc)|(?:ve)|(?:vg)|(?:vi)|(?:vn)|(?:vu)|(?:wf)|(?:ws)|(?:ye)|(?:yt)|(?:za)|(?:zm)|(?:zw))(?:/[^\s()<>]+[^\s`!()\[\]{};:\'".,<>?\xab\xbb\u201c\u201d\u2018\u2019])?)', re.IGNORECASE)
email            = re.compile("([a-z0-9!#$%&'*+\/=?^_`{|.}~-]+@(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?)", re.IGNORECASE)
ip               = re.compile('(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)', re.IGNORECASE)
ipv6             = re.compile('\s*(?!.*::.*::)(?:(?!:)|:(?=:))(?:[0-9a-f]{0,4}(?:(?<=::)|(?<!::):)){6}(?:[0-9a-f]{0,4}(?:(?<=::)|(?<!::):)[0-9a-f]{0,4}(?:(?<=::)|(?<!:)|(?<=:)(?<!::):)|(?:25[0-4]|2[0-4]\d|1\d\d|[1-9]?\d)(?:\.(?:25[0-4]|2[0-4]\d|1\d\d|[1-9]?\d)){3})\s*', re.VERBOSE|re.IGNORECASE|re.DOTALL)
price            = re.compile('[$]\s?[+-]?[0-9]{1,3}(?:(?:,?[0-9]{3}))*(?:\.[0-9]{1,2})?')
hex_color        = re.compile('(#(?:[0-9a-fA-F]{8})|#(?:[0-9a-fA-F]{3}){1,2})\\b')
credit_card      = re.compile('((?:(?:\\d{4}[- ]?){3}\\d{4}|\\d{15,16}))(?![\\d])')
btc_address      = re.compile('(?<![a-km-zA-HJ-NP-Z0-9])[13][a-km-zA-HJ-NP-Z0-9]{26,33}(?![a-km-zA-HJ-NP-Z0-9])')
street_address   = re.compile('\d{1,4} [\w\s]{1,20}(?:street|st|avenue|ave|road|rd|highway|hwy|square|sq|trail|trl|drive|dr|court|ct|park|parkway|pkwy|circle|cir|boulevard|blvd)\W?(?=\s|$)', re.IGNORECASE)
zip_code         = re.compile(r'\b\d{5}(?:[-\s]\d{4})?\b')
po_box           = re.compile(r'P\.? ?O\.? Box \d+', re.IGNORECASE)
ssn              = re.compile('(?!000|666|333)0*(?:[0-6][0-9][0-9]|[0-7][0-6][0-9]|[0-7][0-7][0-2])[- ](?!00)[0-9]{2}[- ](?!0000)[0-9]{4}')

In [ ]:
import regex

In [ ]:
regex.match(r'(?(?=\d)\d+|\w+)', '123abc')

In [ ]:
regex.findall(r'(?(?=\d)\d+|\w+)', '123abc36 7gdh127')

In [ ]:
regex.match(r'(?(?=\d)\d+|\w+)', 'abc123')

In [ ]:
regex.findall(r'(?(?=\d)\d+|\w+)', '123abc')

In [ ]:
# Using re.findall for text
import re

list = ["guru99 get", "guru99 give", "guru Selenium"]
for element in list:
    z = re.match("(g\w+)\W(g\w+)", element)
if z:
    print(z.groups())
    
patterns = ['software testing', 'guru99']
text = 'software testing is fun?'
for pattern in patterns:
    print 'Looking for "%s" in "%s" ->' % (pattern, text),
    if re.search(pattern, text):
        print ('found a match!')
else:
    print ('no match')
abc = 'guru99@google.com, careerguru99@hotmail.com, users@yahoomail.com'
emails = re.findall(r'[\w\.-]+@[\w\.-]+', abc)
for email in emails:
    print (email)


<strong><b><font size="5">Data Science Academy</font></b></strong>

<strong><b><font size="5">Web Scraping e Análise de Dados</font></b></strong>

<strong><b><font size="5">Lab 2</font></b></strong>

<strong><b><font size="4">Web Scraping e Análise Estatística - Avaliando o Desempenho dos Participantes em Corridas de 10 KM com Dados de Sensores RFID</font></b></strong>

![title](imagens/lab2.png)

## Definição do Problema

Você gosta de correr? Corrida é um dos esportes mais praticados em todo mundo, além de uma atividade saudável e prazerosa.

Neste Lab vamos coletar dados reais de participantes de uma corrida de 10 KM realizada em Hillsboro no Oregon, EUA, em junho de 2017. 

Resumo da corrida:

- Pessoas que completaram a corrida: 577
- Homens: 414
- Mulheres: 163

O objetivo desta análise é encontrar:
    
- O tempo médio de conclusão da corrida.
- Se os tempos de chegada dos corredores seguem uma distribuição normal.
- Analisar se há diferença de desempenho entre homens e mulheres de várias faixas etárias.

Os tempos registrados na corrida (bem como outros dados) são bem precisos pois os participantes usaram sensores RFID de onde os dados foram extraídos e disponibilizados na web. Mais detalhes sobre o uso dos sensores RFID aqui:

https://www.hubertiming.com/runners/faq

## Fonte de Dados

Como fonte de dados usaremos o web site <a href="https://www.hubertiming.com/">Huber Timing</a> que contém o resultado das corridas que acontecem nos EUA, bem como a agenda de novos eventos.

Aqui a página com o resultado da corrida que faremos a análise:

https://www.hubertiming.com/results/2017GPTR10K

## Carregando os Pacotes Usados Neste Jupyter Notebook

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook, se necessário.

In [ ]:
# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [ ]:
# Usaremos o pacote Pingouin para algumas das análises estatísticas
# https://pingouin-stats.org/index.html
!pip install -q pingouin

In [ ]:
# Imports
import re
import requests
import matplotlib as m
import numpy as np
import pandas as pd
import pingouin as pg
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
from scipy import stats

# Imports para formatação dos gráficos
sns.set_style('whitegrid')
%matplotlib inline

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

## Web Scraping

In [ ]:
# Definindo a URL com o resultado da corrida
url = 'https://www.hubertiming.com/results/2017GPTR10K'

In [ ]:
# Abre a conexão com a url
html = urlopen(url)

In [ ]:
# Extrai o código HTML
soup = bs(html,'lxml')

In [ ]:
# Extrai o texto do código html
texto = soup.get_text()

In [ ]:
# Vamos visualizar as 10 primeiras linhas
linhas = soup.find_all('tr')
print(linhas[:10])

## Limpeza e Transformação

Agora começa o trabalho pesado. Não fique apenas executando código e compreenda o que está sendo feito. Se tiver dificuldades, pergunte! Temos vários canais de suporte a sua disposição.

Começaremos obtendo todas as linhas da tabela em forma de lista e depois converteremos essa lista em um dataframe para facilitar a manipulação.

In [ ]:
# Loop
for linha in linhas:
    linha_td = linha.find_all('td')

In [ ]:
print(linha_td)

Com expressões regulares em Python, vamos extrair os dados das tags HTML.

In [ ]:
# Lista vazia para receber as linhas
lista_linhas = []

In [ ]:
# Loop
for linha in linhas:
    cells = linha.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '', str_cells))
    lista_linhas.append(clean2)

In [ ]:
print(clean2)

Convertemos a lista em um dataframe e obtemos uma visão rápida das 10 primeiras linhas usando o Pandas.

In [ ]:
# Convertemos a lista em dataframe
df = pd.DataFrame(lista_linhas)

In [ ]:
df.head(10)

Claramente precisamos limpar os dados. Vamos ao trabalho.

In [ ]:
# Dividindo cada linha e separando os dados a cada vírgula
df1 = df[0].str.split(',', expand = True)

In [ ]:
df1.head(10)

Está começando a ficar bonito. Vamos fazer mais algumas limpezas. Leia e compreenda o código. Se tiver dúvidas, pergunte (embora isso não deva ser novidade se você fez o curso gratuito de <a href="https://www.datascienceacademy.com.br/course?courseid=python-fundamentos">Python Fundamentos</a> aqui na DSA).

In [ ]:
# Removemos colchetes de abertura (lado esquerdo) do texto
df1[0] = df1[0].str.strip('[')

In [ ]:
df1.head(10)

In [ ]:
# Extraímos o cabeçalho de todas as tabelas no código HTML
col_labels = soup.find_all('th')

In [ ]:
col_labels

In [ ]:
# Extraímos texto entre tags html dos cabeçalhos de tabela (extraímos o nome)
all_header = []
col_str = str(col_labels)
cleantext2 = bs(col_str, "lxml").get_text()
all_header.append(cleantext2)

In [ ]:
print(all_header)

In [ ]:
# Convertendo a lista de cabeçalhos em um dataframe do pandas
df2 = pd.DataFrame(all_header)

In [ ]:
df2.head()

In [ ]:
# Dividindo a coluna "0" em várias colunas na posição de vírgula para todas as linhas
df3 = df2[0].str.split(',', expand = True)

In [ ]:
df3.head()

In [ ]:
# Concatenando os 2 dataframes
frames = [df3, df1]
df4 = pd.concat(frames)

In [ ]:
df4.head(10)

In [ ]:
# Atribuindo a primeira coluna como cabeçalho
df5 = df4.rename(columns = df4.iloc[0])

In [ ]:
df5.head()

In [ ]:
# Shape
df5.shape

In [ ]:
# Obtendo uma visão geral dos dados para análise
df5.info()

In [ ]:
# Verificando valores ausentes
df5.isnull().sum()

In [ ]:
# Removendo todas as linhas com valores ausentes
df6 = df5.dropna(axis = 0, how = 'any')

In [ ]:
# Descartando o cabeçalho da tabela replicada como a primeira linha no df5
df7 = df6.drop(df6.index[0])

In [ ]:
# Renomeando as colunas [Place e Team] 
df7.rename(columns = {'[Place': 'Place'}, inplace = True)
df7.rename(columns = {' Team]': 'Team'}, inplace = True)

In [ ]:
# Limpeza final dos dados - remoção do colchete de fechamento das células na coluna "Team".
df7['Team'] = df7['Team'].str.strip(']')

In [ ]:
df7.head()

## Análise de Dados

In [ ]:
# Convertendo a coluna "Chip Time" em apenas minutos
# Chip time é o tempo total de corrida medido com a leitura do sensor RFID no selo da camisa do participante
time_list = df7[' Chip Time'].tolist()

In [ ]:
# Visualizando uma amostra de dados
time_list[1:5]

In [ ]:
# Lista para receber o resultado da conversão
time_mins = []

In [ ]:
# Loop para conversão
for i in time_list:
    i = i.strip(' ')
    if len(i)!=7:
        i = '0:' + i
    h, m, s = i.split(':')
    math = (int(h) * 3600 + int(m) * 60 + int(s))/60
    time_mins.append(math)

Convertendo a lista novamente em um dataframe e criamos uma nova coluna ("Runner_mins") como tempo total de corrida, mas agora em minutos.

In [ ]:
# Nova coluna
df7['Runner_mins'] = time_mins

In [ ]:
df7.head()

In [ ]:
df7.dtypes

In [ ]:
# Cálculo de estatísticas para colunas numéricas no dataframe
df7.describe(include = [np.number])

**Análise**:

- O tempo médio de chip para todos os corredores foi de ~ 60 minutos (a média na tabela acima).
- O corredor 10K mais rápido terminou em 36,35 minutos (valor mínimo na tabela acima).
- O corredor mais lento terminou em 101,30 minutos (valor máximo na tabela acima).

Vamos criar um boxplot para sumarizar nossa análise.

In [ ]:
# Plot
ax = sns.boxplot(x = df7["Runner_mins"], palette = "Set3", orient = "v")
plt.ylabel('Chip Time')
plt.xlabel('Runners')
plt.show()

Acima estão as estatísticas de resumo de dados para os corredores mostrados em um boxplot: Máximo, mínimo, médio, primeiro quartil, terceiro quartil, incluindo valores extremos.

Criaremos agora um gráfico de distribuição dos tempos dos chips dos corredores usando a biblioteca seaborn.

In [ ]:
# Plot
x = df7['Runner_mins']
ax = sns.distplot(x, hist = True, kde = True, rug = False, color = 'blue', bins = 25, hist_kws = {'edgecolor':'black'})
plt.show()

A distribuição da variável parece quase normal. Mas vamos realizar um teste estatístico para confirmar.

Um ponto de decisão importante ao trabalhar com uma amostra de dados é usar métodos estatísticos paramétricos ou não paramétricos.

Os métodos estatísticos paramétricos assumem que os dados têm uma distribuição conhecida e específica, geralmente uma distribuição gaussiana (normal). Se uma amostra de dados não é gaussiana, as suposições dos testes estatísticos paramétricos são violadas e métodos estatísticos não paramétricos devem ser utilizados.

Há várias técnicas que você pode usar para verificar se sua amostra de dados se desvia de uma distribuição gaussiana, chamada testes de normalidade.

O pacote stats do SciPy fornece uma função para um dos testes de normalidade mais comuns, o Shapiro-Wilk Test. Usaremos ainda a função normality do pacote Pengouin.

O Shapiro-Wilk é um teste de hipóteses, que testa essas duas hipóteses:

**A hipótese nula deste teste é que a população é normalmente distribuída. Assim, se o valor-p for menor que o nível alfa escolhido (normalmente definido em 0,05), a hipótese nula será rejeitada e há evidências de que os dados testados não são normalmente distribuídos.**

O resultado do teste de Shapiro-Wilk deve ser interpretado com cautela no caso de amostras grandes. 

In [ ]:
# Valor de x
x = df7['Runner_mins']

In [ ]:
# Teste de normalidade com Scipy
stats.shapiro(x)

In [ ]:
# Teste de normalidade com Pingouin
pg.normality(x)

**Análise**:

- A variável não segue uma distribuição normal, uma vez que o valor-p é menor que 0.05 e assim há evidências estatísticas para rejeitar a hipótese nula. A função normality() do pacote Pingouin coloca essa informação na última coluna.

Vamos agora descobrir se houve alguma diferença de desempenho entre homens e mulheres de várias faixas etárias.

In [ ]:
# Separando os dados
f_runners = df7.loc[df7[' Gender'] == ' F']['Runner_mins']
m_runners = df7.loc[df7[' Gender'] == ' M']['Runner_mins']

In [ ]:
# Criaremos os 2 plots em uma só figura para facilitar a comparação
sns.distplot(f_runners, hist = True, kde = True, rug = False, hist_kws = {'edgecolor':'black'}, label = 'Mulher')
sns.distplot(m_runners, hist = False, kde = True, rug = False, hist_kws = {'edgecolor':'black'}, label = 'Homem')
plt.legend()

**Análise**:

- A distribuição indica que as mulheres eram mais lentas que os homens, em média.

In [ ]:
# Computando estatísticas resumidas para homens e mulheres separadamente usando o método groupby ()
g_stats = df7.groupby(" Gender", as_index = True).describe()
g_stats

**Análise**:
    
- O tempo médio de chip para todas as mulheres e homens foi de ~ 66 minutos e ~ 58 minutos, respectivamente, comprovando a análise do gráfico. 

Um boxplot ajuda a comparar as amostras de homens e mulheres.

In [ ]:
# Plot
df7.boxplot(column = 'Runner_mins', by = ' Gender')
plt.ylabel('Chip Time')
plt.suptitle("")

Nossa fonte de dados possui resultados de diversas corridas. Material em abundância para você praticar seus conhecimentos em análise de dados.

# Fim